# Create protein expression atlas

In [1]:
import pandas as pd
import mysql.connector
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from ipywidgets import IntProgress
from IPython.display import display

Loosely based on source code from Tine Claeys (see Reference folder)

In [2]:
conn = mysql.connector.connect(user='root', password='password', host='127.0.0.1', port='3306',database='expression_atlas_cells')
mycursor = conn.cursor(buffered = True)

# check the connection
if conn.is_connected():
    print("connection succesfull")
else:
    print("no connection")

connection succesfull


# Load all the data

Reformat so a matrix is generated with columns: assay_id, quantification, uniprot_id
- Load peptide_to_assay
- Load peptide_to_protein
- Load protein information from protein table
- Filter: Only get proteotypic peptides (peptide -> 1 protein)
- Filter: Drop contaminant proteins according to the cRAP database
- Filter: Only get proteins with more than 2 peptide relations

In [3]:
projectsql = "SELECT PXD_accession FROM project"
projectData = pd.read_sql_query(projectsql, conn)
projectData.head()

,PXD_accession
0,PXD000533
1,PXD004280
2,PXD002842
3,PXD003594
4,PXD008996


In [6]:
projectData.PXD_accession.nunique()

63

In [3]:
# NOTE: Loading the table takes 5 minutes

assaysql = "SELECT assay_id, peptide_id, quantification FROM peptide_to_assay"
assayData = pd.read_sql_query(assaysql, conn)
assayData.head()

,assay_id,peptide_id,quantification
0,30960,110730450,2.0
1,30961,110730450,2.0
2,30995,110730450,1.0
3,31006,110730450,1.0
4,31007,110730450,1.0


In [11]:
# Before 18/02/2023: 1943

assayData.assay_id.nunique()

4414

In [ ]:
meta = pd.read_csv("../Metadata/unified_metadata.csv")
assayData[assayData.assay_id.isin(meta.assay_id.tolist())]

Get all peptide to protein relations

In [12]:
pepsql = "SELECT peptide_to_protein.peptide_id, peptide_to_protein.uniprot_id FROM peptide_to_protein"
pepData = pd.read_sql_query(pepsql, conn)
pepData.head()

,peptide_id,uniprot_id
0,122045961,A0A024QZ42
1,120203377,A0A024QZX5
2,122405701,A0A024R1R8
3,132052084,A0A024R214
4,129311384,A0A024R7E8


In [13]:
# Before 18/02/23 addition of data: 424682

pepData.shape

(496188, 2)

Get sequence length for all proteins in the database

In [14]:
seqsql = "SELECT * FROM protein WHERE length IS NOT NULL"
seqData = pd.read_sql_query(seqsql, conn)
seqData["length"] = pd.to_numeric(seqData['length'], errors = "coerce")

print(seqData.shape)

seqData.head()

(16893, 4)


,uniprot_id,description,length,sequence
0,A0A024RBG1,Diphosphoinositol polyphosphate phosphohydrola...,181.0,MMKFKPNQTRTYDREGFKKRAACLCFRSEQEDEVLLVSSSRYPDQW...
1,A0A075B6H7,Probable non-functional immunoglobulin kappa v...,116.0,MEAPAQLLFLLLLWLPDTTREIVMTQSPPTLSLSPGERVTLSCRAS...
2,A0A075B6I1,Immunoglobulin lambda variable 4-60,120.0,MAWTPLLLLFPLLLHCTGSLSQPVLTQSSSASASLGSSVKLTCTLS...
3,A0A075B6I3,Probable non-functional immunoglobulin lambda ...,123.0,MALTPLLLLLLSHCTGSLSRPVLTQPPSLSASPGATARLPCTLSSD...
4,A0A075B6L6,Probable non-functional T cell receptor beta v...,115.0,MGTRLLCWAALCLLGADHTGAGVSQTPSNKVTEKGKDVELRCDPIS...


In [15]:
# There are 11756 unreviewed protein identifiers from the TREMBL database. These could be linked to the reviewed SwissProt entry through mapping.
# https://www.uniprot.org/help/gene_centric_isoform_mapping
# Local file: UP000005640_9606.idmapping

print('# TREMBL protein identifiers: ', pepData[~pepData.uniprot_id.isin(seqData.uniprot_id)].uniprot_id.nunique())
print('% peptides ided with TREMBL: {:.2f}%'.format(pepData[~pepData.uniprot_id.isin(seqData.uniprot_id)].shape[0]/pepData.shape[0] *100))

# TREMBL protein identifiers:  11756
% peptides ided with TREMBL: 5.04%


Select proteotypic peptides (peptide with 1 peptide to protein relation)

In [16]:
# Before 18/02/2023: 422679

proteotypicData = pepData.groupby("peptide_id").filter(lambda x: len(x) == 1)
print(proteotypicData.shape)
proteotypicData.head()

(494039, 2)


,peptide_id,uniprot_id
0,122045961,A0A024QZ42
1,120203377,A0A024QZX5
2,122405701,A0A024R1R8
3,132052084,A0A024R214
4,129311384,A0A024R7E8


Drop non human proteins (contaminant removal) <br>
- Check the protein ids that contain strange symbols ([-_|])
- Convert them to standard uniprot identifiers
- Check all these identifiers are human proteins

In [17]:
# Check for protein identifiers that do not match the uniprot id regex.
proteotypicData = proteotypicData.copy()
contaminants = proteotypicData[proteotypicData.uniprot_id.str.contains(pat = r"[-_|]")].uniprot_id.unique().tolist()
contaminants[:5]

['ADH1_YEAST', 'ADH1_YEAST|', 'ALBU_BOVIN', 'ALBU_BOVIN|', 'ALDOA_RABIT']

Some are human proteins, yet the function that parsed the protein identifiers failed to parse it correctly... <br>
Search manually for these and change the protein identifiers accordingly. Then check through the cRAP database whether common contaminants are included

In [18]:
# From the cRAP https://www.thegpm.org/crap/index.html database, common contaminant identifiers were extracted
CRAP = pd.read_csv("CRAP.tsv", sep="\t")
CRAP["uniprot_id"] = CRAP.Description.apply(lambda x: x[1:7])

In [19]:
# All the failed uniprot ids are CRAP so delete them from the proteotypicData data structure

uniprot_parser_failed = {"AMYS_HUMAN|":     "/",        # Not found in uniprot
                         "ANT3_HUMAN|":     "P01008",   # Antithrombin-III
                         "B2MG_HUMAN|":     "P61769",   # Beta-2-microglobulin
                         "BID_HUMAN|":      "P55957",   # BH3-interacting domain death agonist
                         "CATG_HUMAN|":     "P08311",   # Cathepsin G
                         "GELS_HUMAN|":     "P06396",   # Gelsolin
                         "HBB_HUMAN|":      "P68871",   # Hemoglobin subunit beta
                         "H-INV":           "/",        # Not found
                         "IGF2_HUMAN|":     "P01344",   # Insulin-like growth factor II
                         'K1C9_HUMAN|':     'P35527',   # Keratin, type I cytoskeletal 9
                         'K1H2_HUMAN|':     "Q14532",   # Keratin, type I cuticular Ha2
                         'K1H4_HUMAN|':     "O76011",   # Keratin, type I cuticular Ha4
                         'K1H5_HUMAN':      "Q92764",   # Keratin, type I cuticular Ha5
                         'K1H5_HUMAN|':     "Q92764",   # Keratin, type I cuticular Ha5,
                         'K1H8_HUMAN|':     "O76015",   # Keratin, type I cuticular Ha8
                         'K1HA_HUMAN|':     'O76009',   # Keratin, type I cuticular Ha3-I
                         "K1H6_HUMAN|":     "O76013",   # Keratin, type I cuticular Ha6
                         "K1HB_HUMAN|":     "Q14525",   # Keratin, type I cuticular Ha3-II
                         "KCRM_HUMAN|":     "P06732",   # Creatine kinase M-type
                         "KRHB4_HUMAN|":    "Q9NSB2",   # Keratin, type II cuticular Hb4
                         'KRHB3_HUMAN|':    "P78385",   # Keratin, type II cuticular Hb3
                         'KRHB4_HUMAN':     "Q9NSB2",   # Keratin, type II cuticular Hb4,
                         'KRHB5_HUMAN|':    "P78386",   # Keratin, type II cuticular Hb5
                         'LALBA_HUMAN|':    "P00709",   # Alpha-lactalbumin
                         'LEP_HUMAN|':      "P41159",   # Leptin
                         "LYSC_HUMAN|":     "P61626",   # Lysozyme C
                         'NEDD8_HUMAN|':    "Q15843",   # NEDD8
                         'NQO2_HUMAN|':     "P16083",   # Ribosyldihydronicotinamide dehydrogenase [quinone]
                         "P01045-1":        "P01045",   # canonical sequence of Kininogen-2, a BOVINE protein
                         "P02535-1":        "P02535",   # Canonical sequence of Keratin, the MOUSE protein
                         "P08730-1":        "P08730",   # Another MOUSE keratin protein
                         "P13646-1":        "P13646",   # Human keratin
                         'PPIA_HUMAN':      "P62937",   # Peptidyl-prolyl cis-trans isomerase A
                         'PPIA_HUMAN|':     "P62937",   # Peptidyl-prolyl cis-trans isomerase A
                         "RASH_HUMAN|":     "P01112",   # GTPase HRas
                         'RS27A_HUMAN|':    "P62979",   # Ubiquitin-40S ribosomal protein S27a
                         'TAU_HUMAN':       "P10636",   # Microtubule-associated protein tau
                         'TNFA_HUMAN|':     "P01375",   # Tumor necrosis factor
                         "UBE2C_HUMAN|":    "O00762"}   # Ubiquitin-conjugating enzyme E2 C

# Convert the non-standard identifiers
for key, parser_id in uniprot_parser_failed.items():
    if parser_id not in CRAP.uniprot_id.unique().tolist():
        print(key)

AMYS_HUMAN|
H-INV
P01045-1
P02535-1
P08730-1
P13646-1


In [20]:
def status_protein(protein_id):
    if protein_id in list(uniprot_parser_failed.values()):
        return 'contaminant'
    if protein_id in CRAP.uniprot_id.unique().tolist():
        return 'contaminant'
    else:
        return 'identification'

proteotypicData["parsed_uniprot"] = proteotypicData.uniprot_id.apply(lambda x: uniprot_parser_failed[x] if x in list(uniprot_parser_failed.keys()) else x)
proteotypicData["status"] = proteotypicData.parsed_uniprot.apply(status_protein)

In [21]:
# Reformat
proteotypicData.drop("uniprot_id", axis = 1, inplace = True)
proteotypicData.rename(columns={"parsed_uniprot":"uniprot_id"},inplace=True)

# Merge with uniprot SwissProt file containing sequence and length information
proteotypicData = pd.merge(proteotypicData, seqData, on = "uniprot_id")
proteotypicData = proteotypicData[~(proteotypicData.status == "contaminant")]

Select proteins which have more than 2 proteotypic peptides

In [22]:
print(proteotypicData.shape)
proteins = proteotypicData.groupby("uniprot_id").filter(lambda x: len(x) > 2)
print(proteins.shape)

(468838, 6)
(465311, 6)


Merge assays containing spectral counts and proteins

In [23]:
protData = pd.merge(assayData, proteins, on = "peptide_id").sort_values(["assay_id", "uniprot_id"])
print(protData.shape)
protData.head()

(12882034, 8)


,assay_id,peptide_id,quantification,uniprot_id,status,description,length,sequence
56525,30960,110734028,5.0,A0AV96,identification,RNA-binding protein 47 (RNA-binding motif prot...,593.0,MTAEDSTAAMSSDSAAGSSAKVPEGVAGAPNEAALLALMERTGYSM...
85567,30960,110736528,2.0,A0AVT1,identification,Ubiquitin-like modifier-activating enzyme 6 (U...,1052.0,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...
111025,30960,110737579,2.0,A0AVT1,identification,Ubiquitin-like modifier-activating enzyme 6 (U...,1052.0,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...
141112,30960,110739628,3.0,A0AVT1,identification,Ubiquitin-like modifier-activating enzyme 6 (U...,1052.0,MEGSEPVAAHQGEEASCSSWGTGSTNKNLPIMSTASVEIDDALYSR...
56253,30960,110733975,3.0,A0FGR8,identification,Extended synaptotagmin-2 (E-Syt2) (Chr2Syt),921.0,MTANRDAALSSHRHPGCAQRPRTPTFASSSQRRSAFGFDDGNFPGL...


In [24]:
protData.assay_id.nunique()
protData = protData.iloc[:,:4]
del protData["peptide_id"]

---
---

# Quantify with NSAF (unpooled)

Split data per assay

In [ ]:
assays = protData['assay_id'].unique()
DataFramaDict = {elem: pd.DataFrame for elem in assays}

f = IntProgress(min=0, max=len(DataFramaDict))
display(f)

for key in DataFramaDict.keys():
    DataFramaDict[key] = protData[:][protData["assay_id"] == key]
    f.value += 1

In [199]:
import copy
DataFramaDict2 = copy.deepcopy(DataFramaDict)

Calculate NSAF score for each protein per assay

In [200]:
f = IntProgress(min=0, max = len(DataFramaDict2))
display(f)

for count, key in enumerate(DataFramaDict2.keys()):
    sumSaf = 0
    assay = DataFramaDict2[key]
    assay.pop("assay_id")

    #calculate sum of spectral counts for each protein
    grouped = DataFramaDict2[key].groupby("uniprot_id").sum().reset_index()
    seqAddedDF = pd.merge(grouped, seqData, on = "uniprot_id")
    seqAddedDF.insert(loc = 2, column = 'SAF', value = 0)
    seqAddedDF.insert(loc = 3, column = 'NSAF', value = 0)
    
    #Calculate SAF score for each protein by dividing sum of spectral counts by protein length
    for index, row in seqAddedDF.iterrows():
        saf = row['quantification']/row['length']
        seqAddedDF.loc[index, 'SAF'] = saf
        # calculate sum of SAF scores in assay
        sumSaf += saf

    # Calculate NSAF score by normalizing each SAF score
    seqAddedDF["NSAF"] = seqAddedDF["SAF"] / sumSaf
    
    del seqAddedDF['length']
    del seqAddedDF['quantification']
    del seqAddedDF['SAF']
    seqAddedDF.insert(loc = 0, column = 'assay_id', value = key)
    DataFramaDict2[key] = seqAddedDF

    f.value += 1

IntProgress(value=0, max=4414)

In [209]:
f = IntProgress(min=0, max = len(DataFramaDict2))
display(f)

proteinData = []

for key in DataFramaDict2.keys():
    proteinData.append(DataFramaDict2[key])
    f.value += 1

proteinData = pd.concat(proteinData, ignore_index=True)

IntProgress(value=0, max=4414)

In [ ]:
df = proteinData

In [210]:
proteinData.assay_id.nunique()

4414

In [52]:
#write NSAF proteome to file
df = df[["assay_id", "uniprot_id", "NSAF"]]
df.to_hdf(path_or_buf = 'proteome_nsaf_3.h5', key = "df", mode = "w")

In [4]:
df = pd.read_hdf('proteome_nsaf_3.h5')

# Quantify with NSAF (pooled)

Add pool_id metadata for pooling the fractions <br>
Then fuse the fraction tables and calculate NSAF together

In [25]:
sqlquery = "SELECT assay_id, filename FROM assay"
assay_meta = pd.read_sql_query(sqlquery, conn)
assay_meta = assay_meta[assay_meta.assay_id.isin(df.assay_id)]
print(assay_meta.shape[0])

4414


In [26]:
meta = pd.read_csv("../Metadata/unified_metadata.csv")
meta.shape

(4322, 20)

In [33]:
no_pool = meta[meta.pool_id.isin([0])].assay_id.tolist()

#list in list of pools
pools = meta[~meta.assay_id.isin(no_pool)].groupby(["PXD_accession", "pool_id"]).apply(lambda x: list(x.assay_id)).tolist()

print("Whole lysate samples: {}\nFractions: {} ({} assays collapsed)".format (len(no_pool), len(pools), sum([len(pool) for pool in pools])))
print("Total: {}".format(len(no_pool)+len(pools)))

Whole lysate samples: 389
Fractions: 232 (3933 assays collapsed)
Total: 621


In [ ]:
f = IntProgress(min=0, max = len(no_pool))
f2 = IntProgress(min=0, max = len(pools))
display(f)

DataFrameDict_no_pool = {elem: pd.DataFrame for elem in no_pool}
for key in DataFrameDict_no_pool.keys():
    DataFrameDict_no_pool[key] = protData[:][protData["assay_id"] == key]
    f.value += 1

display(f2)

DataFrameDict_pooled = {pool[0]: pd.DataFrame for pool in pools}
for pool in pools:
    DataFrameDict_pooled[pool[0]] = protData[:][protData.assay_id.isin(pool)]
    f2.value += 1

DataFrameDict_no_pool.update(DataFrameDict_pooled)

In [37]:
import copy
DataFramaDict_pooling = copy.deepcopy(DataFrameDict_no_pool)

In [39]:
f = IntProgress(min=0, max = len(DataFramaDict_pooling))
display(f)

for count, key in enumerate(DataFramaDict_pooling.keys()):
    sumSaf = 0
    assay = DataFramaDict_pooling[key]
    assay.pop("assay_id")

    #calculate sum of spectral counts for each protein
    grouped = DataFramaDict_pooling[key].groupby("uniprot_id").sum().reset_index()
    seqAddedDF = pd.merge(grouped, seqData, on = "uniprot_id")
    seqAddedDF.insert(loc = 2, column = 'SAF', value = 0)
    seqAddedDF.insert(loc = 3, column = 'NSAF', value = 0)
    
    #Calculate SAF score for each protein by dividing sum of spectral counts by protein length
    for index, row in seqAddedDF.iterrows():
        saf = row['quantification']/row['length']
        seqAddedDF.loc[index, 'SAF'] = saf
        # calculate sum of SAF scores in assay
        sumSaf += saf

    # Calculate NSAF score by normalizing each SAF score
    seqAddedDF["NSAF"] = seqAddedDF["SAF"] / sumSaf
    
    del seqAddedDF['length']
    del seqAddedDF['quantification']
    del seqAddedDF['SAF']
    seqAddedDF.insert(loc = 0, column = 'assay_id', value = key)
    DataFramaDict_pooling[key] = seqAddedDF
    f.value += 1

IntProgress(value=0, max=621)

In [42]:
f = IntProgress(min=0, max = len(DataFramaDict_pooling))
display(f)

proteinData_pooled = []

for key in DataFramaDict_pooling.keys():
    proteinData_pooled.append(DataFramaDict_pooling[key])
    f.value += 1

proteinData_pooled = pd.concat(proteinData_pooled, ignore_index=True)

IntProgress(value=0, max=621)

In [43]:
proteinData_pooled.assay_id.nunique()

621

In [48]:
proteinData_pooled = proteinData_pooled[["assay_id", "uniprot_id", "NSAF"]]

In [49]:
#write NSAF proteome to file
proteinData_pooled.to_hdf(path_or_buf = 'proteome_nsaf_pooled_3.h5', key = "df", mode = "w")

In [50]:
df2 = pd.read_hdf('proteome_nsaf_pooled_3.h5')

---
---